# gpu/ram/drive

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Nov 26 07:57:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import gc
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# import

In [16]:
import random
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

# outlier 확인

In [9]:
train= pd.read_csv('drive/MyDrive/dacon/data/train.csv')
test= pd.read_csv('drive/MyDrive/dacon/data/test.csv')
train2 = pd.read_csv('drive/MyDrive/dacon/data/train_ver2.csv', encoding='cp949')

In [ ]:
# train_input_path, train_target_path = 'drive/MyDrive/dacon/data/train_input/*.csv', 'drive/MyDrive/dacon/data/train_target/*.csv'
# test_input_path, test_target_path = 'drive/MyDrive/dacon/data/test_input/*.csv', 'drive/MyDrive/dacon/data/test_target/*.csv'
# train = make_dataset(train_input_path, train_target_path)
# test = make_dataset(test_input_path, test_target_path)

In [12]:
train_under40_t=train2[train2['predicted_weight_g']<40]
train_under40_f=train2[train2['predicted_weight_g']>=40]

In [13]:
train_under40_f.describe()

,DAT,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,predicted_weight_g,Case
count,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000,5712.000000
mean,21.823529,25.506663,70.738094,461.108406,1.655590,693.754835,8394.406614,4801.228507,55462.923371,1935.643490,23350.607579,1567.688313,17359.450579,8304.560310,96172.981530,88.200554,14.239496
std,4.138009,3.308737,9.640867,102.718547,1.237962,623.684213,8590.778937,7339.305063,79979.037519,3215.856858,39290.942744,2824.869392,32660.758765,7215.359243,81276.406039,36.880783,7.174738
min,9.000000,14.961667,20.641667,314.883333,0.000000,-653.260000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.075834,1.000000
25%,20.000000,23.943333,61.574583,382.783333,0.467327,123.400000,1740.440000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17329.280575,57.681391,9.000000
50%,23.000000,25.854285,73.120000,434.958333,1.417632,762.570000,4614.000000,0.000000,0.000000,0.000000,2706.425400,0.000000,156.650000,9242.350000,90027.419900,79.755601,14.000000
75%,25.000000,26.865000,78.145417,512.391667,2.632536,1248.650000,13403.390000,9253.679575,104623.897400,1952.664000,22339.800000,2272.208250,21152.057875,13327.835750,159351.799725,114.892809,19.000000
max,27.000000,39.842373,84.945000,1360.933333,5.146290,3144.130000,42855.230000,18564.600000,292014.969800,9928.800000,200396.280000,9399.000000,190016.450000,23637.426100,292014.969800,164.150300,28.000000


In [21]:
abs(train2.iloc[3370:3380,2:])

,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,predicted_weight_g,Case
3370,27.443333,62.096667,408.516667,0.387035,769.0,5383.0,18255.1900,72466.9161,1012.7376,3837.4812,2805.6015,7205.9000,22073.5291,83510.2973,2.106971,6
3371,27.476667,61.383333,447.066667,0.385749,769.0,6152.0,18255.1900,90722.1061,1164.9792,5002.4604,2951.2860,10157.1860,22371.4552,105881.7525,2.106971,6
3372,27.638333,60.565000,453.400000,0.384616,769.0,6921.0,18255.1900,108977.2961,1323.8400,6326.3004,3101.6700,13258.8560,22680.7000,128562.4525,2.106971,6
3373,27.603333,61.771667,404.966667,0.383738,769.0,7690.0,18255.1900,127232.4861,1495.9392,7822.2396,3264.5860,16523.4420,23015.7152,151578.1677,2.106971,6
3374,27.505000,60.616667,480.816667,0.385774,769.0,8459.0,18255.1900,145487.6761,1659.7644,9482.0040,3419.6695,19943.1115,23334.6239,174912.7916,2.106971,6
3375,27.384000,74.100000,410.120000,0.388644,769.0,9228.0,138371.2461,7116.4300,8793.6072,688.3968,18570.8575,1372.2540,165735.7108,9177.0808,2.106971,6
3376,25.590000,60.220000,409.483333,0.394163,769.0,9997.0,17744.6635,24861.0935,2128.0728,2816.4696,3223.8570,4596.1110,23096.5933,32273.6741,2.106971,6
3377,24.471667,63.376667,409.450000,0.387544,769.0,10766.0,16831.9040,41692.9975,2616.2388,5432.7084,2302.7550,6898.8660,21750.8978,54024.5719,2.106971,6
3378,23.703333,64.280000,409.033333,0.388473,769.0,11535.0,8678.9505,50371.9480,1509.1776,6941.8860,971.2300,7870.0960,11159.3581,65183.9300,2.106971,6
3379,21.620000,68.926667,408.183333,0.393056,0.0,11535.0,0.0000,50371.9480,0.0000,6941.8860,0.0000,7870.0960,0.0000,65183.9300,2.106971,6


In [15]:
for i in range(0,len(train_under40_t)/24):
  train_under40_t.iloc[24*i:24*i+24]
  

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,predicted_weight_g,Case
0,0,00:00,25.3,81.835,536.016667,1.407439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.167719,1


# 전처리

In [175]:
def preprocessing(input_path):
  all_input_list = sorted(glob.glob(input_path))
  train = pd.DataFrame()
  for datapath in all_input_list:
    data = pd.read_csv(datapath) # 이게 지금 28일치 * 24시
    data['obs_time'] = data.index % 24 # 시간통일
    data = abs(data)
    col_list = data.columns
    for i in range(0,len(col_list)):
      col = col_list[i]    
      if '누적' in col :
        data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()
    data.to_csv(datapath,index=False)
    train = pd.concat([train,data])
  return train

In [ ]:
b = preprocessing('drive/MyDrive/dacon/data/train_input/*.csv')

In [ ]:
a = preprocessing('drive/MyDrive/dacon/data/test_input/*.csv')

In [186]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]
            time_list = day['obs_time'].unique()
            for col in col_list :
                for time in time_list :
                    value = day[day['obs_time']==time][col].iloc[0]
                    x[col+str(time)] = value
            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)
    return train

In [187]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data/train_input/*.csv', 'drive/MyDrive/dacon/data/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data/test_input/*.csv', 'drive/MyDrive/dacon/data/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [188]:
train

,내부온도관측치0,내부온도관측치1,내부온도관측치2,내부온도관측치3,내부온도관측치4,내부온도관측치5,내부온도관측치6,내부온도관측치7,내부온도관측치8,내부온도관측치9,...,일간누적총광량16,일간누적총광량17,일간누적총광량18,일간누적총광량19,일간누적총광량20,일간누적총광량21,일간누적총광량22,일간누적총광량23,DAT,predicted_weight_g
0,25.300000,25.680357,25.273333,25.355000,25.391667,25.375000,26.436667,26.960000,27.128334,27.366666,...,210911.3740,230142.8960,249374.4180,268605.9400,269376.2938,269376.2938,269376.2938,269376.2938,1,0.167719
1,25.806666,25.981034,25.583334,25.476667,25.431666,25.406666,27.105000,27.765000,28.025000,28.196667,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,2,0.181787
2,26.098334,26.313559,25.878333,25.794999,25.660001,25.601667,27.388333,28.068334,28.228333,27.823333,...,210727.0981,229958.6201,249190.1421,268421.6641,269399.5381,269399.5381,269399.5381,269399.5381,3,0.265921
3,25.438333,25.783051,25.321666,25.211667,25.275000,25.245000,26.893333,27.470000,26.641667,26.379661,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,4,0.423650
4,25.470000,25.462712,25.401667,25.331667,25.461667,25.410000,26.391666,26.698334,26.806666,26.938333,...,210838.9167,230070.4387,249301.9607,268533.4827,269511.3567,269511.3567,269511.3567,269511.3567,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,30.368333,30.362711,30.768333,30.645000,30.946666,30.826666,31.135000,31.525000,32.485000,30.155000,...,79507.0460,95162.5780,103428.9048,103438.1871,103438.1871,103438.1871,103438.1871,103438.1871,24,64.875499
780,23.680000,23.717241,23.818333,23.660000,23.658333,23.561666,23.730000,24.688333,25.783333,25.746667,...,155738.5910,171394.1230,179463.2992,179469.4874,179469.4874,179469.4874,179469.4874,179469.4874,25,74.002614
781,24.028333,24.075862,24.208333,24.101667,24.013333,23.900000,23.938333,24.725000,25.946667,25.996667,...,155505.1534,171160.6854,179423.2327,179432.5150,179432.5150,179432.5150,179432.5150,179432.5150,26,76.342275
782,24.050000,23.985000,24.058334,23.980000,23.966667,23.920000,23.950000,24.631667,25.600000,25.545000,...,155528.0655,171183.5975,179360.7245,179373.1009,179373.1009,179373.1009,179373.1009,179373.1009,27,82.621245


In [198]:
train.to_csv('drive/MyDrive/dacon/data/train_누적합수정.csv',index=False)
test.to_csv('drive/MyDrive/dacon/data/test_누적합수정.csv',index=False)